# PaddleOCR-VL 合同解析实践项目本 Notebook 记录了一个完整的 PaddleOCR-VL 多语言合同解析流程，涵盖项目背景、目录结构、环境准备、数据构造、模型调用、测试验证以及最终效果展示。通过逐步演示，帮助你快速复现并扩展到自有业务场景。

## 项目目标- 自动解析中英文合同并生成结构化 Markdown/JSON；- 抽取甲乙方、合同金额、签署日期等关键信息；- 提供 CLI 与 Gradio Demo，方便集成与展示；- 附带单元测试与可复现合成样本，便于撰写实践文章。

## 系统架构概览````text输入合同 (PDF/图片)    │    ├──> PaddleOCRVL  (文档预处理 + 版面分析 + VLM 识别)    │    ├── 结构化 Markdown/JSON    │    ├──> contract_extraction.extract_contract_fields    │       └── 正则 + 表格解析    │    ├──> 关键信息 (party_a, party_b, amount, dates...)    │    ├──> CLI / Notebook / Gradio 展示    │    └──> 数据导出 (Markdown、JSON、可选入库)````

## 目录结构说明````textcontract_paddleocr_vl/├── README.md├── app.py                       # 命令行入口├── contract_extraction.py       # 关键信息抽取├── contract_ocr_pipeline.py     # PaddleOCR-VL 调用封装├── contract_ocr_demo.ipynb      # 本 Notebook├── data/│   └── synthetic_contracts/     # 合成合同样本├── generate_synthetic_contracts.py├── gradio_app.py└── tests/    └── test_contract_extraction.py````

In [ ]:
from pathlib import Path
import sys

project_root = Path.cwd()
sys.path.append(str(project_root))
for path in sorted(project_root.glob('**/*')):
    if path.is_file():
        print(path)


## 环境准备1. 推荐使用 `conda activate paddle` 激活虚拟环境；2. 安装依赖：   ```bash   python -m pip install paddlepaddle-gpu==3.2.1 -i https://www.paddlepaddle.org.cn/packages/stable/cu126/   python -m pip install -U "paddleocr[doc-parser]" gradio pillow pytest   ```3. 若需 vLLM/SGLang 后端，执行 `paddleocr install_genai_server_deps vllm` 并启动 `paddleocr genai_server`。

In [ ]:
import platform, sys, importlib

print('Python', sys.version)
print('Platform', platform.platform())
for pkg in ['paddleocr', 'gradio', 'pytest']:
    status = '已安装' if importlib.util.find_spec(pkg) else '未安装'
    print(f'{pkg}: {status}')


## 运行单元测试 (保障抽取逻辑可用)使用内置示例验证 `extract_contract_fields` 的正确性。

In [ ]:
import pytest
pytest.main(['tests/test_contract_extraction.py'])


## 第一步：生成合成合同样本脚本会生成中英文 Markdown/PNG 合同。设置 `create_images=True` 以生成可直接用于 OCR 的 PNG。

In [ ]:
from shutil import rmtree
from contract_paddleocr_vl.generate_synthetic_contracts import create_samples

samples_dir = project_root / 'data' / 'notebook_samples'
if samples_dir.exists():
    rmtree(samples_dir)
samples_dir.mkdir(parents=True, exist_ok=True)
create_samples(samples_dir, count=4, create_images=True)
sorted(str(p) for p in samples_dir.iterdir())


## 第二步：执行 OCR 流程并抽取关键信息`run_contract_ocr` 会自动完成文档预处理、版面分析与 VLM 推理，并写出结果。

In [ ]:
from contract_paddleocr_vl.contract_ocr_pipeline import run_contract_ocr

ocr_outputs = project_root / 'data' / 'notebook_outputs'
if ocr_outputs.exists():
    rmtree(ocr_outputs)
ocr_outputs.mkdir(parents=True, exist_ok=True)
result = run_contract_ocr(samples_dir, output_dir=ocr_outputs)
result['fields']


## 输出检查：生成的 JSON / Markdown 文件每份合同会落地 JSON 与 Markdown，便于后续入库或审核。

In [ ]:
sorted(str(p) for p in ocr_outputs.glob('*'))


## 解析第一份合同的 Markdown展示第一份样本的 Markdown 结果。

In [ ]:
from IPython.display import Markdown as IPyMarkdown

markdown_segments = result.get('markdown', [])
if markdown_segments:
    first = markdown_segments[0]
    if isinstance(first, dict):
        md_text = first.get('markdown_text') or first.get('markdown', '')
    else:
        md_text = first
    IPyMarkdown(md_text)
else:
    print('未获取到 Markdown 结果，可能是 OCR 未成功执行。')


## 命令行调用示例```bashpython app.py --input data/notebook_samples/CN-001.png --output-dir tmp_outputs```也可在项目外层执行 `python -m contract_paddleocr_vl.app ...`。

## Gradio Web Demo```bashpython gradio_app.py```上传合同即可查看解析结果，可部署为内部工具。

## 总结与下一步- PaddleOCR-VL 在单一流程中完成预处理、版面分析、多模态识别；- 结合正则与 Markdown 结构，快速抽取合同关键信息；- 后续可扩展：  1. 丰富 `_FIELD_PATTERNS`，覆盖更多语言与模板；  2. 将结果写入向量数据库，实现合同问答；  3. 封装为 Flask/FastAPI 服务；  4. 在不同硬件与推理后端上做性能评估。